In [ ]:
import os
import pinecone
import dotenv

# load environment
load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")

pinecone.init(api_key=pinecone_api_key, environment="gcp-starter")
